# 한국어 전처리


> 솔트룩스 AI Labs NLP파트 김성현 (seonghyunkim@saltlux.com)



## 0. Introduction

한국어에서의 다양한 전처리 방식들을 실습합니다.

* Basic
 - 가장 기초적인 전처리
 - html tag 제거
 - 숫자 제거
 - Lowercasing
 - "@%*=()/+ 와 같은 punctuation 제거
* Spell check
 - 사전 기반의 오탈자 교정
 - 줄임말 원형 복원 (e.g. I'm not happy -> I am not happy)
* Part-of-Speech
 - 형태소 분석
 - Noun, Adjective, Verb, Adverb만 학습에 사용
* Stemming
 - 형태소 분석 이후 동사 원형 복원
* Stopwords
 - 불용어 제거
* Negation
 - [논문](https://dl.acm.org/doi/pdf/10.5555/2392701.2392703)
 - 부정 표현에 대한 단순화 (e.g. I'm not happy -> I'm sad)
 - 한국어에서의 적용이 어려워, 추후 추가할 예정


먼저 실습을 위해 한국어 wikipedia 문서를 다운받도록 하겠습니다.

In [ ]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1EcJpRTEdGVaYhbLE1otE5iCifj_kW1_4" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1EcJpRTEdGVaYhbLE1otE5iCifj_kW1_4" -o wiki_20190620.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    857      0 --:--:-- --:--:-- --:--:--   857
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  476M    0  476M    0     0  18.0M      0 --:--:--  0:00:26 --:--:-- 27.7M


## 1. Basic Preprocessing

In [ ]:
# 한국어 위키 데이터 load
data = open('/home/kpkim/문서/kpkim/wiki_20190620.txt', 'r', encoding='utf-8')
lines = data.readlines()

In [ ]:
for i in range(0, 10):
    print(lines[i])

한국어 문장 분리 라이브러리 중, 가장 성능이 좋은 tokenizer 중 하나인 kss를 설치합니다.

In [ ]:
!pip install kss

### 문장 분절

In [ ]:
import kss

sentence_tokenized_text = []
for i, line in enumerate(lines):
    if i > 100:     # 전체 wikipedia 문서는 사이즈가 크므로, 일부만 테스트.
        break
    line = line.strip()
    for sent in kss.split_sentences(line):
        sentence_tokenized_text.append(sent.strip())

이제 `sentence_tokenized_text`에 문장 단위로 분리된 corpus가 저장되었습니다.

### punctuation 처리, 제거

In [ ]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [ ]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [ ]:
def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text.strip()

In [ ]:
cleaned_corpus = []
for sent in sentence_tokenized_text:
    cleaned_corpus.append(clean_punc(sent, punct, punct_mapping))

In [ ]:
for i in range(0, 10):
    print(cleaned_corpus[i])

### 정규식을 활용한 정제

In [ ]:
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

In [ ]:
basic_preprocessed_corpus = clean_text(cleaned_corpus)

In [ ]:
for i in range(0, 10):
    print(basic_preprocessed_corpus[i])

## 2. Spell check

띄어쓰기 검사로는 [한국어 띄어쓰기 검사 라이브러리](https://github.com/haven-jeon/PyKoSpacing)를 사용하고,   
맞춤법 검사로는 [한국어 맞춤법 검사 라이브러리](https://github.com/ssut/py-hanspell)와, [논문](https://link.springer.com/chapter/10.1007/978-3-030-12385-7_3)에서 사용되었던 외래어 사전을 사용하겠습니다.   
반복되는 이모티콘이나 자소는 이 [라이브러리](https://github.com/lovit/soynlp)를 이용해 필터링 하겠습니다.   

먼저 띄어쓰기 검사기를 설치하겠습니다.

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
from pykospacing import spacing
spacing("김형호영화시장분석가는'1987'의네이버영화정보네티즌10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다.")

다음으로 맞춤법 검사기를 설치하겠습니다.

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
from hanspell import spell_checker
 
sent = "대체 왜 않돼는지 설명을 해바"
spelled_sent = spell_checker.check(sent)
checked_sent = spelled_sent.checked
 
print(checked_sent)

다음으로는 데이터에서 반복되는 이모티콘이나 자모를 normalization을 위한 라이브러리를 설치하도록 하겠습니다.

In [ ]:
!pip install soynlp

In [ ]:
from soynlp.normalizer import *
print(repeat_normalize('와하하하하하하하하하핫', num_repeats=2))

마지막으로 외래어 사전을 다운로드 받겠습니다.

In [ ]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1RNYpLE-xbMCGtiEHIoNsCmfcyJP3kLYn" -o confused_loanwords.txt

In [ ]:
lownword_map = {}
lownword_data = open('/home/kpkim/문서/kpkim/confused_loanwords.txt', 'r', encoding='utf-8')

lines = lownword_data.readlines()

for line in lines:
    line = line.strip()
    miss_spell = line.split('\t')[0]
    ori_word = line.split('\t')[1]
    lownword_map[miss_spell] = ori_word

In [ ]:
def spell_check_text(texts):
    corpus = []
    for sent in texts:
        spaced_text = spacing(sent)
        spelled_sent = spell_checker.check(sent)
        checked_sent = spelled_sent.checked
        normalized_sent = repeat_normalize(checked_sent)
        for lownword in lownword_map:
            normalized_sent = normalized_sent.replace(lownword, lownword_map[lownword])
        corpus.append(normalized_sent)
    return corpus

In [ ]:
spell_preprocessed_corpus = spell_check_text(basic_preprocessed_corpus)
print(spell_preprocessed_corpus)

## 3. Part-of-Speech 

Python 기반의 형태소 분석기 중, 성능이 가장 좋은 것 중 하나인 카카오의 [Khaiii](https://github.com/kakao/khaiii)를 사용하겠습니다.

In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

In [ ]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

test_sents = ["나도 모르게 사버렸다.", "행복해야해!", "내가 안 그랬어!", "나는 사지 않았어.", "하나도 안 기쁘다.", "상관하지마", "그것 좀 가져와"]

for sent in test_sents:
    for word in api.analyze(sent):
        for morph in word.morphs:
            print(morph.lex + '/' + morph.tag)
    print('\n')


In [ ]:
significant_tags = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'VX', 'MAG', 'MAJ', 'XSV', 'XSA']

def pos_text(texts):
    corpus = []
    for sent in texts:
        pos_tagged = ''
        for word in api.analyze(sent):
            for morph in word.morphs:
                if morph.tag in significant_tags:
                    pos_tagged += morph.lex + '/' + morph.tag + ' '
        corpus.append(pos_tagged.strip())
    return corpus

In [ ]:
pos_tagged_corpus = pos_text(spell_preprocessed_corpus)

In [ ]:
for i in range(0, 30):
    print(pos_tagged_corpus[i])

## 4. Stemming

동사를 원형으로 복원하도록 하겠습니다.
규칙은 다음과 같습니다.

1. NNG|NNP|NNB + XSV|XSA --> NNG|NNP|NNB + XSV|XSA + 다
2. NNG|NNP|NNB + XSA + VX --> NNG|NNP + XSA + 다
3. VV --> VV + 다
4. VX --> VX + 다

In [ ]:
p1 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XS.')
p2 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XSA [가-힣A-Za-z0-9]+/VX')
p3 = re.compile('[가-힣A-Za-z0-9]+/VV')
p4 = re.compile('[가-힣A-Za-z0-9]+/VX')

In [ ]:
def stemming_text(text):
    corpus = []
    for sent in text:
        ori_sent = sent
        mached_terms = re.findall(p1, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        
        mached_terms = re.findall(p2, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                if tag != 'VX':
                    modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p3, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p4, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        corpus.append(ori_sent)
    return corpus

In [ ]:
stemming_corpus = stemming_text(pos_tagged_corpus)

In [ ]:
for i in range(0, 30):
    print(stemming_corpus[i])

## 5. Stopwords

불용어는 도메인에 맞춰서 다양하게 구축될 수 있습니다.   

In [ ]:
stopwords = ['데/NNB', '좀/MAG', '수/NNB', '등/NNB']

In [ ]:
def remove_stopword_text(text):
    corpus = []
    for sent in text:
        modi_sent = []
        for word in sent.split(' '):
            if word not in stopwords:
                modi_sent.append(word)
        corpus.append(' '.join(modi_sent))
    return corpus

In [ ]:
removed_stopword_corpus = remove_stopword_text(stemming_corpus)

In [ ]:
for i in range(0, 30):
    print(removed_stopword_corpus[i])